## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-25-04-00-00 +0000,wsj,India Is on a Himalayan Building Spree to Prep...,https://www.wsj.com/world/asia/india-is-on-a-h...
1,2025-12-25-04-00-00 +0000,wsj,How an Elusive Banker Allegedly Kept North Kor...,https://www.wsj.com/world/asia/how-an-elusive-...
2,2025-12-25-03-52-51 +0000,nypost,Elementary school teacher falls and dies after...,https://nypost.com/2025/12/24/world-news/itali...
3,2025-12-25-03-36-03 +0000,nypost,Joyride ends in disaster as driver smashes int...,https://nypost.com/2025/12/24/world-news/late-...
4,2025-12-25-03-32-10 +0000,bbc,Trump-backed candidate wins in knife-edge Hond...,https://www.bbc.com/news/articles/cyvgz1jn368o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,christmas,22
39,trump,20
96,new,12
84,killed,9
392,ukraine,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
195,2025-12-24-10-30-00 +0000,wsj,Capital Account: President Trump’s social-medi...,https://www.wsj.com/economy/central-banking/wh...,66
24,2025-12-25-00-41-15 +0000,bbc,Where is Santa now? Trump answers kids' Christ...,https://www.bbc.com/news/videos/cqxqvd0z5gno?a...,56
112,2025-12-24-17-22-59 +0000,nypost,DOJ seeks Christmas-week volunteers to speed u...,https://nypost.com/2025/12/24/us-news/doj-plea...,52
138,2025-12-24-15-15-38 +0000,nypost,Ukraine agrees to demilitarized economic zone ...,https://nypost.com/2025/12/24/us-news/ukraine-...,52
42,2025-12-24-23-07-16 +0000,nypost,Passenger explodes at Alaska Airlines staff fo...,https://nypost.com/2025/12/24/us-news/passenge...,52


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
195,66,2025-12-24-10-30-00 +0000,wsj,Capital Account: President Trump’s social-medi...,https://www.wsj.com/economy/central-banking/wh...
42,52,2025-12-24-23-07-16 +0000,nypost,Passenger explodes at Alaska Airlines staff fo...,https://nypost.com/2025/12/24/us-news/passenge...
69,49,2025-12-24-21-10-21 +0000,nypost,New details revealed in gruesome NYC garbage t...,https://nypost.com/2025/12/24/us-news/new-deta...
126,39,2025-12-24-16-15-06 +0000,nyt,Zelensky Open to Pulling Back Troops in Easter...,https://www.nytimes.com/2025/12/24/world/europ...
166,33,2025-12-24-12-26-16 +0000,nypost,Explosion in Moscow kills three people — inclu...,https://nypost.com/2025/12/24/world-news/explo...
39,30,2025-12-24-23-23-47 +0000,nypost,Trump-backed conservative Nasry Asfura declare...,https://nypost.com/2025/12/24/us-news/trump-ba...
90,30,2025-12-24-19-26-29 +0000,nypost,DOJ unearths 1 million more possible Jeffrey E...,https://nypost.com/2025/12/24/us-news/doj-unea...
52,30,2025-12-24-22-31-33 +0000,nypost,Mom shot dead in hail of bullets inside daught...,https://nypost.com/2025/12/24/us-news/mom-shot...
91,29,2025-12-24-19-21-49 +0000,nypost,California law could force 2M homeowners to ri...,https://nypost.com/2025/12/24/us-news/californ...
116,28,2025-12-24-16-53-16 +0000,nypost,Afghan DC National Guard shooting suspect Rahm...,https://nypost.com/2025/12/24/us-news/afghan-d...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
